# Algoritmia
## Práctica Obligatoria 3
### Curso 2022 - 2023
###### Programación dinámica
---
 

#### Autores:
* <Alvar San Martin>
* <>

---
Resuelva la siguiente práctica.

Importe las librerías que desees
**Recuerda**: 
* Solamente puedes utilizar librerías nativas (https://docs.python.org/es/3.7/library/index.html).
  * <sub><sup>_Importe las librerías que desees._</sup></sub>
* Se recomienda utilizar un entorno con la versión 3.7 (`conda create -n <nombre_entorno> python=3.7`). Más información en https://conda.io/projects/conda/en/latest/user-guide/tasks/manage-environments.html.
* Las funciones que importes no son "gratis", cada una tendrá una complejidad temporal y espacial que se tendrá que tener en cuenta.
* Las funciones que crees, han de estar en una celda que comience por `#testeable` para que se importe en los test.

**Entrega**
* Poner el nombre del fichero como: `<apellidosPrimerAlumno>_<apellidosSegundoAlumno>_voraz.ipynb`.
    * En caso de que el fichero no tenga ese nombre, la entrega tendrá una penalización de **2 puntos**
* Verificar que la entrega no está corrupta.
    * En caso de que la entrega está corrupta, se evaluará con **0 puntos**.
* Ambos alumnos tendrán que hacer la entrega.
    * En caso de que uno no la haga se evaluará como **No presentado**, si las entregas son diferentes tendrá cada una una penalización de **2 puntos** y se corregirán por separado.

In [135]:
#testeable
# Imports utiles
from math import inf

In [136]:
# Solo para los print
from tabulate import tabulate

## Datos Perdidos S.L.

In [162]:
#testeable
class BaseDatosCorrupta:
    """
    Clase BaseDatosCorrupta que contiene una cadena de texto corrupta.

    Parameters
    ----------
    cadenaCorrupta : str
        Cadena de texto corrupta, no debe de reemplazarse.
    """

    def __init__(self, cadenaCorrupta):
        self.cadenaCorrupta = cadenaCorrupta
        self.cadenaEncontrada = None
        self.completa = None

    def recuperacion_datos(self, seq, character_null=""):
        """
        Dadas la cadena seq devuelve una que es subsecuencia de seq y cadenaCorrupta que tiene la longitud máxima de
        todas las subsecuencias comunes.

        Parameters
        ----------
        seq : str
        character_null: str
            Carácter a ignorar
        """
        # Caso limite, una de las cadenas esta vacia
        if len(seq) == 0 or len(self.cadenaCorrupta) == 0:
            self.cadenaEncontrada = ""
            return ''
        
        # filtrado de caracteres nulos
        seq = ''.join(l for l in seq if l not in character_null)

        # Matriz dinamica n+1 x m+1 iniciada a 0s
        # asumimos que la cadena orginial es mas larga por comodidad
        lens = [[0] * (len(self.cadenaCorrupta)+1) for i in range(len(seq)+1)]

        for i_org, letra_org in enumerate(seq):
            # Debug
            # test = "Ɛ" + seq
            # print("Comparacion:",letra_org, letra_comp)
            # print(tabulate([[test[i]]+v for i,v in enumerate(lens)], headers="Ɛ"+self.cadenaCorrupta, tablefmt='fancy_grid'))
            # print("********")
            for i_comp, letra_comp in enumerate(self.cadenaCorrupta):
                
                # Si los caracteres coinciden
                if letra_org == letra_comp:
                    # Se aumenta el valor de longitud de cadena a partir de ese momento 
                    # tener en cuenta el offset Ɛ para la col 1 fila 1
                    lens[i_org+1][i_comp+1] = lens[i_org][i_comp] + 1
                else:
                    # Si no coincide, la casilla de abajo derecha es el maximo de abajo o derecha
                    # (Ha habido una coincidencia previa)
                    lens[i_org+1][i_comp+1] = max(lens[i_org+1][i_comp], lens[i_org][i_comp+1])
        
        # Debug
        # test = "Ɛ" + seq
        # print("Comparacion:",letra_org, letra_comp)
        # print(tabulate([[test[i]]+v for i,v in enumerate(lens)], headers="Ɛ"+self.cadenaCorrupta, tablefmt='fancy_grid'))
        # print("********")
        ###
                
        
        # Lectura desde la matriz
        subcadena = ""
        fila = len(seq)
        col = len(self.cadenaCorrupta)

        while(fila > 0 and col > 0):
            # print("----")
            #print("indices: ",fila,col)
            #print("valores: ",seq[fila-1], self.cadenaCorrupta[col-1])
            if seq[fila-1] == self.cadenaCorrupta[col-1]:
                # Si son iguales movimiento diagonal
                fila, col = fila-1, col-1
                subcadena += self.cadenaCorrupta[col]
                #print("Coincidencia")
                #print("Cadena:",subcadena)
            else:
                # se busca el maximo
                if lens[fila-1][col] > lens[fila][col-1]:
                    fila-=1
                else:
                    col-=1
        

        self.cadenaEncontrada = subcadena[::-1]

        self.completa = self.cadenaEncontrada == seq

        
        return subcadena[::-1]

    def get_cadenaEncontrada(self):
        """
        Devuelve los datos encontrados tras la llamada a recuperacion_datos.

        Returns
        -------
        str
        
        """
        return self.cadenaEncontrada

    def encontradaCadenaCompleta(self):
        """
        Indica si la llamada a recuperacion_datos encontró la secuencia de datos 

        Returns
        -------
        bool
        
        """
        return self.completa


In [185]:
bd_str = ""
base = BaseDatosCorrupta(bd_str)
res = base.recuperacion_datos("","")
res

In [187]:
base.cadenaEncontrada

### Ministerio de Transporte, Movilidad y Agenda Urbana

In [140]:
#testeable
class RedComarcas:
    """
    Red de comarcas para conectar con carreteras.

    Parameters
    ----------
    nombreComarcas : list
        Lista con los nombres de las comarcas que aparecen en el grafo
    grafo : dict
        Diccionario que contiene un grafo dirigido, la clave es una tupla con el par ("comarcaOrigen", "comarcaDestino")
        el valor de cada clase es el peso del arco, ej: {("c1", "c2"): 1}
    """

    def __init__(self, nombreComarcas, grafo):
        self.nombreComarcas = nombreComarcas
        self.grafo = grafo
        self.distancias = None
        self.caminos = None

    def calcularRutas(self):
        """
        Calcula la ruta distancias optimas entre cualquier comarca.
        En base al algoritmo de Floyd-Warshall
        """
        # matriz de distancias
        distancias = [[inf] * (len(self.nombreComarcas)) for i in range(len(self.nombreComarcas))]
        
        # matriz de caminos para la reconstruccion
        caminos = [[None] * (len(self.nombreComarcas)) for i in range(len(self.nombreComarcas))]

        # Diagonales
        for i in range(0,len(self.nombreComarcas)):
            distancias[i][i] = 0
            caminos[i][i] = i

        for nodos, peso in self.grafo.items():
            org,dest = self.nombreComarcas.index(nodos[0]),self.nombreComarcas.index(nodos[1])
            distancias[org][dest] = peso
            caminos[org][dest] = org
            
        #print(tabulate([[self.nombreComarcas[i]]+v for i,v in enumerate(distancias)], headers=self.nombreComarcas, tablefmt='fancy_grid'))
        #print(tabulate([[self.nombreComarcas[i]]+v for i,v in enumerate(caminos)], headers=self.nombreComarcas, tablefmt='fancy_grid'))
        
        ncomarcas = len(self.nombreComarcas)

        # k = iteracion
        # i = filas
        # j = columnas
        for k in range(0,ncomarcas):
            for i in range(0,ncomarcas):
                for j in range(0,ncomarcas):
                    if distancias[i][j] > distancias[i][k] + distancias[k][j]:
                        distancias[i][j] = distancias[i][k] + distancias[k][j]
                        caminos[i][j] = caminos[k][j]

        #print(tabulate([[self.nombreComarcas[i]]+v for i,v in enumerate(distancias)], headers=self.nombreComarcas, tablefmt='fancy_grid'))
        #print(tabulate([[self.nombreComarcas[i]]+[self.nombreComarcas[x] for x in v] for i,v in enumerate(caminos)], headers=self.nombreComarcas, tablefmt='fancy_grid'))
        
        self.distancias = distancias
        self.caminos = caminos
        return distancias
        

    def distancia(self, origen, destino):
        """
        Devuelve la distancia del camino mínimo ente origen y destino.
        Si no hay camino devuelve None.

        Parameters
        ----------
        string:origen
        string:destino

        Returns
        -------
        int
        """
        if origen not in self.nombreComarcas or destino not in self.nombreComarcas:
            return None
        origen = self.nombreComarcas.index(origen)
        destino = self.nombreComarcas.index(destino)
        print("distancia",origen,destino)
        return self.distancias[origen][destino]

    def camino(self, origen, destino):
        """
        Devuelve en una lista de nodos el camino mínimo entre origen y destino.
        Si no hay camino devuelve None.

        Parameters
        ----------
        string:origen
        string:destino

        Returns
        -------
        list
            Lista con el nombre de las comarcar a cruzar para llegar al destino, ejemplo:
            Para ir de c1 a c3 pasando por c2: ["c1", "c2", "c3"]
            Para ir de c2 a c3: ["c2", "c3"]

            
        procedure Path(org, dest)
        if prev[org][dest] = null then
            return []
        path ← [dest]
        while org ≠ dest
            dest ← prev[org][dest]
            path.prepend(dest)
        return path
        """
        if origen not in self.nombreComarcas or destino not in self.nombreComarcas:
            return None

        origen = self.nombreComarcas.index(origen)
        destino = self.nombreComarcas.index(destino)

        if self.caminos[origen][destino] is None:
            return []       

        actual = destino
        
        camino = [destino]

        while actual != origen:
            # se biusca como llegar al origen desde el destino
            # se llega cuando el un nodo apunta a si mismo para llegar al destino
            
            print(self.nombreComarcas[actual],self.nombreComarcas[origen])
            actual = self.caminos[origen][actual]
            camino.append(actual)
        print([self.nombreComarcas[i] for i in camino[::-1]])
        return [self.nombreComarcas[i] for i in camino[::-1]]


### Caso de ejemplo 1

In [141]:
import unittest

In [142]:
bd_str = "0poeiarhgpaeoLimuxirghpaeiguAnahapeiguhsenighIsabeluseñpgiohsegrpighusdñifRomanghnp688732017202331666heogisdkfjgh9"
bd = BaseDatosCorrupta(bd_str)

class TestBaseDatos(unittest.TestCase):

    def test_cadenaEncontrada1(self):
        bd.recuperacion_datos("Ana Isabel Roman", "")
        self.assertEqual(bd.get_cadenaEncontrada(), "AnaIsabelRoman")

    def test_cadenaEncontrada2(self):
        bd.recuperacion_datos("ES6887320871720233166658", "")
        self.assertEqual(bd.get_cadenaEncontrada(), "688732017202331666")

    def test_contieneCadenaComun1(self):
        bd.recuperacion_datos("Ana Isabel Roman", " ")
        self.assertTrue(bd.encontradaCadenaCompleta())

        bd.recuperacion_datos("Ana Isabel Roman", "")
        self.assertFalse(bd.encontradaCadenaCompleta())

    def test_ignorarCaracterExistene1(self):
        """
        No tiene utilidad, pero es posible ignorar caracteres dentro de la cadena que se busca
        """
        bd.recuperacion_datos("Ana Isabel Roman", "A")
        self.assertEqual(bd.get_cadenaEncontrada(), "naIsabelRoman")

### Caso de ejemplo 2

In [143]:
nombreComarcas = ["Palencia","Burgos","Valladolid","Segovia"]
grafo = {
    ("Palencia", "Segovia"): 1,
    ("Segovia", "Palencia"): 2,
    ("Palencia", "Burgos"): 8,
    ("Burgos", "Valladolid"): 3,
    ("Valladolid", "Palencia"): 4,
    ("Segovia", "Burgos"): 10,
    ("Segovia", "Valladolid"): 9
}

# Origen, destino, red, distancia
pruebas = [
    ["Palencia", "Palencia", ["Palencia"], 0],
    ["Palencia", "Burgos", ["Palencia", "Burgos"], 8],
    ["Palencia", "Valladolid", ["Palencia", "Segovia", "Valladolid"], 10],
    ["Palencia", "Segovia", ["Palencia", "Segovia"], 1],
    ["Burgos", "Palencia", ["Burgos", "Valladolid", "Palencia"], 7],
    ["Burgos", "Burgos", ["Burgos"], 0],
    ["Burgos", "Valladolid", ["Burgos", "Valladolid"], 3],
    ["Burgos", "Segovia", ["Burgos", "Valladolid", "Palencia", "Segovia"], 8],
    ["Valladolid", "Palencia", ["Valladolid", "Palencia"], 4],
    ["Valladolid", "Burgos", ["Valladolid", "Palencia", "Burgos"], 12],
    ["Valladolid", "Valladolid", ["Valladolid"], 0],
    ["Valladolid", "Segovia", ["Valladolid", "Palencia", "Segovia"], 5],
]

red = RedComarcas(nombreComarcas, grafo)
red.calcularRutas()

red.camino(origen="Burgos",destino="Segovia")

class TestRedComarcas(unittest.TestCase):

    def test_caminos_correctos1(self):
        for prueba in pruebas:
            self.assertEqual(red.camino(prueba[0], prueba[1]), prueba[2])

    def test_distancias_correctos1(self):
        for prueba in pruebas:
            self.assertEqual(red.distancia(prueba[0], prueba[1]), prueba[3])

╒════════════╤════════════╤══════════╤══════════════╤═══════════╕
│            │   Palencia │   Burgos │   Valladolid │   Segovia │
╞════════════╪════════════╪══════════╪══════════════╪═══════════╡
│ Palencia   │          0 │        8 │          inf │         1 │
├────────────┼────────────┼──────────┼──────────────┼───────────┤
│ Burgos     │        inf │        0 │            3 │       inf │
├────────────┼────────────┼──────────┼──────────────┼───────────┤
│ Valladolid │          4 │      inf │            0 │       inf │
├────────────┼────────────┼──────────┼──────────────┼───────────┤
│ Segovia    │          2 │       10 │            9 │         0 │
╘════════════╧════════════╧══════════╧══════════════╧═══════════╛
╒════════════╤════════════╤══════════╤══════════════╤═══════════╕
│            │   Palencia │   Burgos │   Valladolid │   Segovia │
╞════════════╪════════════╪══════════╪══════════════╪═══════════╡
│ Palencia   │          0 │        0 │              │         0 │
├─────────

In [144]:
for prueba in pruebas:
    red.distancia(prueba[0], prueba[1])

distancia 0 0
distancia 0 1
distancia 0 2
distancia 0 3
distancia 1 0
distancia 1 1
distancia 1 2
distancia 1 3
distancia 2 0
distancia 2 1
distancia 2 2
distancia 2 3


In [145]:
if __name__ == "__main__":
    unittest.main(argv=['first-arg-is-ignored'], exit=False)

......
----------------------------------------------------------------------
Ran 6 tests in 0.013s

OK


['Palencia']
Burgos Palencia
['Palencia', 'Burgos']
Valladolid Palencia
Segovia Palencia
['Palencia', 'Segovia', 'Valladolid']
Segovia Palencia
['Palencia', 'Segovia']
Palencia Burgos
Valladolid Burgos
['Burgos', 'Valladolid', 'Palencia']
['Burgos']
Valladolid Burgos
['Burgos', 'Valladolid']
Segovia Burgos
Palencia Burgos
Valladolid Burgos
['Burgos', 'Valladolid', 'Palencia', 'Segovia']
Palencia Valladolid
['Valladolid', 'Palencia']
Burgos Valladolid
Palencia Valladolid
['Valladolid', 'Palencia', 'Burgos']
['Valladolid']
Segovia Valladolid
Palencia Valladolid
['Valladolid', 'Palencia', 'Segovia']
distancia 0 0
distancia 0 1
distancia 0 2
distancia 0 3
distancia 1 0
distancia 1 1
distancia 1 2
distancia 1 3
distancia 2 0
distancia 2 1
distancia 2 2
distancia 2 3


##### **Tests**

Para probar que tu solución pasa los tests. Utilice el comando:

```bash
$ python tests-py3<version de python> <mi notebook>
```


Los tests necesitan de las librerías numpy y pandas

```bash
$ pip install numpy pandas
```

###### Explicación de los tests
* `test_ej1_cadenaEncontrada`: Comprueba que encuentra la cadena.
* `test_ej1_ignorarCaracterExistente`: Comprueba que es capaz de ignorar un caracter existente.
* `test_ej1_limite`: Comprueba casos límite, como que la cadena esté vacía, que no encuentre ninguna cadena, que el caracter a ignorar no esté en la cadena, etc.
* `test_ej2_caminos_correctos`: Comprueba que encuentra los caminos correctos entre dos ciudades.
* `test_ej2_distancias_correctos`: Comprueba que encuentra las distancias correctas entre dos ciudades.
* `test_ej2_ciudad_inexistente`: Comprueba que el algoritmo no falla en caso de que intente encontrar un camino entre dos ciudades que no existen.

---

### **Informe**
Contesta a las siguientes preguntas.

#### **Complejidad**
1. Método `BaseDatosCorrupta.recuperacion_datos`
    * **Complejidad temporal**: La complejidad temporal es O(n*m) siendo m y n las dos cadenas a comparar. Es simple de intuir ya que generamos y recorremos una vez una tabla de n * m elementos.
    * **Complejidad espacial**: La complejidad espacial depende de la tabla, por lo que tambien es de O(n*m)
2. Método `RedComarcas.calcularRutas`
    * **Complejidad temporal**: En este algoritmo nos encontramos tablas de n*n siendo n el numero de nodos, que se recorren k veces, que tambien depende del numero de nodos por lo que los 3 bucles anidados nos dan n\*n\*n operaciones, la complejidad sera O(n^3).
    * **Complejidad espacial**: Usamos 2 tablas de n*n una para los pesos de las rutas y la otra para hacer los recorridos, tendremos 2n^2 elementos, que eliminando las constrantes nos queda como O(n^2)

#### **Datos Perdidos S.L.**

* Describa brevemente como sería un método de fuerza bruta (que no se debería de haber implementado en la práctica) para resolver este problema.

El metodo de fuerza bruta es conocido como Naive LCS y consiste en generar todas las posibles subsecuencias y quedarse con la mas larga, esto implica recorrer varias veces cada cadena buscando todas las posibles combinaciones ordenadas. La complejidad seria radicalmente mas alta, O(n*2^n) al ser un metodo combinatorio.

* ¿Cómo serviría de utilidad el algoritmo implementado en un software de control de versiones como git, subversion, bazaar...?

Buscando las partes iguales de un texto podemos encontrar tambien las que no lo son, es decir, los cambios. De este modo se podrian localizar los cambios de los archivos y evitar modificar todos los archivos de un proyecto.

#### **Ministerio de Transporte, Movilidad y Agenda Urbana**

* ¿La solución es óptima (minimiza siempre el valor pedido) o es aproximada (encuentra un mínimo local)?

La solución es optima, se comprueban todos los posibles caminos, es una de las razonas por las que es tan costoso de ejecutar.

* Existiendo el camino a->b con coste 10 y el camino óptimo a->c->b con coste 4+5, describe como tu algoritmo encuentra este segundo.

En una primera iteración, donde busca los caminos de *a* encuentra a->b como el camino mas corto, pero una vez llega a *c* y ha leido que su coste a->c + c->b es menor que a->b, cambia la tabla de costes y de direcciones para que ahora costes[a][b] valga 9 y la ruta[a][b] apunte a *c* y no a *a*. 